# Reddit

### Import modules

In [1]:
import ciso8601
import requests #Pushshift accesses Reddit via an url so this is needed
import json #JSON manipulation
import datetime
import time
import jsonpickle
import praw
import sys
import os

In [ ]:
## First Approach

### Authentication

In [ ]:
with open('credentials/reddit_credentials') as f:
  credentials2 = json.load(f)

In [ ]:
reddit = praw.Reddit(client_id=credentials2["client_id"], client_secret=credentials2["client_secret"], user_agent=credentials2['user_agent'])
fName = 'data/reddit10000.txt' # We'll store the tweets in a text file.



hot_posts = reddit.subreddit('Covid').top(limit=10000)
posts = 0
comments = 0
with open(fName, 'w') as f:
    for post in hot_posts:
        post.comments.replace_more(limit=None)
        #print("Title",post.title)
        f.write(jsonpickle.encode(post.title, unpicklable=False) +'\n')
        #print("Self Test",post.selftext)
        f.write(jsonpickle.encode(post.selftext, unpicklable=False) +'\n')
        posts = posts +1
        for top_level_comment in post.comments:
            #print("C:",top_level_comment.body)
            f.write(jsonpickle.encode(top_level_comment.body, unpicklable=False) +'\n')
            comments = comments + 1


print ("Downloaded {0} posts and {1} comments, Saved to {2}".format(posts,comments, fName))


In [ ]:
reddit = praw.Reddit(client_id=credentials2["client_id"], client_secret=credentials2["client_secret"], user_agent=credentials2['user_agent'])
fName = 'data/reddit1.txt' # We'll store the tweets in a text file.



hot_posts = reddit.subreddit('Covid').top(limit=10000)
posts = 0
comments = 0
with open(fName, 'w') as f:
    for post in hot_posts:
        post.comments.replace_more(limit=None)
        #print("Title",post.title)
        f.write(jsonpickle.encode(post.title, unpicklable=False) +'\n')
        #print("Self Test",post.selftext)
        f.write(jsonpickle.encode(post.selftext, unpicklable=False) +'\n')
        posts = posts +1
        for top_level_comment in post.comments:
            #print("C:",top_level_comment.body)
            f.write(jsonpickle.encode(top_level_comment.body, unpicklable=False) +'\n')
            comments = comments + 1


print ("Downloaded {0} posts and {1} comments, Saved to {2}".format(posts,comments, fName))



## Second Approach

### Using Pushshift Module to extract Submissions Data from Reddit via Python

### Pushshift get URL

In [62]:
#Adapted from this https://gist.github.com/dylankilkenny/3dbf6123527260165f8c5c3bc3ee331b
#This function builds an Pushshift URL, accesses the webpage and stores JSON data in a nested list
def getPushshiftData(query, after, before, sub):
    #Build URL
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    #Print URL to show user
    print(url)
    #Request URL
    r = requests.get(url)
    #Load JSON data from webpage into data variable
    data = json.loads(r.text)
    #return the data element which contains all the submissions data
    return data['data']

### Extract key information from Submissions

We want key data for further analysis including:
* Submission Title
* Author
* Submission post ID
* Upload Time
* Text


In [63]:
#This function will be used to extract the key data points from each JSON result
def collectSubData(subm,id):
    #subData was created at the start to hold all the data which is then added to our global subStats dictionary.
    subData = {} #list to store data points
    subData['id'] = id
    subData['internal_id'] = subm['id']
    subData['publication_date'] = int(subm['created_utc'])#1520561700.0
    subData['source'] = "Reddit"
    subData['author'] = subm['author']
    subData['title'] = subm['title']
    subData['text'] = subm['selftext']
    #Put all data points into a tuple and append to subData
    #Create a dictionary entry of current submission data and store all data related to it
    subStats[id] = subData

### Search Settings

In [64]:
#Create your timestamps and queries for your search URL
#https://www.unixtimestamp.com/index.php > Use this to create your timestamps
after = "1577836800" #int(time.mktime(ciso8601.parse_datetime(date_after).timetuple())) #Submissions after this timestamp
before = "1607040000" #int(time.mktime(ciso8601.parse_datetime(date_after).timetuple())) #Submissions after this timestamp
query = "COVID" #Keyword(s) to look for in submissions
sub = "COVID" #Which Subreddit to search in

#subCount tracks the no. of total submissions we collect
subCount = 0
#subStats is the dictionary where we will store our data.
subStats = {}

In [65]:
# We need to run this function outside the loop first to get the updated after variable
data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date
id = 0
while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    for submission in data:
        collectSubData(submission, id)
        id+=1
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
    after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
    data = getPushshiftData(query, after, before, sub)

print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=COVID&size=1000&after=1577836800&before=1607040000&subreddit=COVID
100
2020-03-26 20:59:00
https://api.pushshift.io/reddit/search/submission/?title=COVID&size=1000&after=1585274340&before=1607040000&subreddit=COVID
100
2020-04-01 21:53:37
https://api.pushshift.io/reddit/search/submission/?title=COVID&size=1000&after=1585796017&before=1607040000&subreddit=COVID
100
2020-04-08 04:55:42
https://api.pushshift.io/reddit/search/submission/?title=COVID&size=1000&after=1586339742&before=1607040000&subreddit=COVID
100
2020-04-12 07:54:14
https://api.pushshift.io/reddit/search/submission/?title=COVID&size=1000&after=1586696054&before=1607040000&subreddit=COVID
100
2020-04-17 12:47:55
https://api.pushshift.io/reddit/search/submission/?title=COVID&size=1000&after=1587145675&before=1607040000&subreddit=COVID
100
2020-04-23 15:58:54
https://api.pushshift.io/reddit/search/submission/?title=COVID&size=1000&after=1587675534&before=1607040000&subr

### Check your Submission Extraction was successful

In [66]:
num_submissions = len(subStats)
print(str(num_submissions) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0]["title"])
print("Last entry is:")
print(list(subStats.values())[-1]["title"])

3398 submissions have added to list
1st entry is:
Coronavirus disease named Covid-19
Last entry is:
Covid Help - Testing - How To Properly Quarantine?


### Save data to JSON file

In [67]:
def updateSubs_file():
    ts = str(round(time.time() * 1000))
    file = "data/reddit_" + str(num_submissions) +"_" + ts + ".JSON"
    with open(file, 'w') as file:
        json.dump(subStats,file)
    print("JSON created")

updateSubs_file()

JSON created


## Second approach
Taken from: https://github.com/Watchful1/Sketchpad/blob/master/postDownloader.py
https://www.reddit.com/r/pushshift/comments/ffzyjj/get_all_comments_from_specific_subreddit/


In [3]:
import requests
from datetime import datetime
import traceback
import time
import json
import sys


username = ""  # put the username you want to download in the quotes
subreddit = "CoronaVirus_ITALIA"  # put the subreddit you want to download in the quotes
# English Coronavirus
# Spanish covidmx
# Italian CoronaVirus_ITALIA

# leave either one blank to download an entire user's or subreddit's history
# or fill in both to download a specific users history from a specific subreddit

filter_string = None
if username == "" and subreddit == "":
	print("Fill in either username or subreddit")
	sys.exit(0)
elif username == "" and subreddit != "":
	filter_string = f"subreddit={subreddit}"
elif username != "" and subreddit == "":
	filter_string = f"author={username}"
else:
	filter_string = f"author={username}&subreddit={subreddit}"

url = "https://api.pushshift.io/reddit/{}/search?limit=1000&sort=desc&{}&before="

start_time = datetime.utcnow()


def downloadFromUrl(filename, object_type):
	print(f"Saving {object_type}s to {filename}")

	count = 0
	handle = open(filename, 'w')
	previous_epoch = int(start_time.timestamp())
	while True:
		new_url = url.format(object_type, filter_string)+str(previous_epoch)
		json_text = requests.get(new_url, headers={'User-Agent': "Post downloader by /u/Watchful1"})
		time.sleep(1)  # pushshift has a rate limit, if we send requests too fast it will start returning error messages
		try:
			json_data = json_text.json()
		except json.decoder.JSONDecodeError:
			time.sleep(1)
			continue

		if 'data' not in json_data:
			break
		objects = json_data['data']
		if len(objects) == 0:
			break

		for object in objects:
			previous_epoch = object['created_utc'] - 1
			count += 1
			if object_type == 'comment':
				try:
					handle.write(str(object['score']))
					handle.write(" : ")
					handle.write(datetime.fromtimestamp(object['created_utc']).strftime("%Y-%m-%d"))
					handle.write("\n")
					handle.write(object['body'].encode(encoding='ascii', errors='ignore').decode())
					handle.write("\n-------------------------------\n")
				except Exception as err:
					print(f"Couldn't print comment: https://www.reddit.com{object['permalink']}")
					print(traceback.format_exc())
			elif object_type == 'submission':
				if object['is_self']:
					if 'selftext' not in object:
						continue
					try:
						handle.write(str(object['score']))
						handle.write(" : ")
						handle.write(datetime.fromtimestamp(object['created_utc']).strftime("%Y-%m-%d"))
						handle.write("\n")
						handle.write(object['selftext'].encode(encoding='ascii', errors='ignore').decode())
						handle.write("\n-------------------------------\n")
					except Exception as err:
						print(f"Couldn't print post: {object['url']}")
						print(traceback.format_exc())

		print("Saved {} {}s through {}".format(count, object_type, datetime.fromtimestamp(previous_epoch).strftime("%Y-%m-%d")))

	print(f"Saved {count} {object_type}s")
	handle.close()


downloadFromUrl("data/reddit_italian_posts.txt", "submission")
downloadFromUrl("data/reddit_italian_comments.txt", "comment")

Saving submissions to data/reddit_italian_posts.txt
Saved 100 submissions through 2020-11-24
Saved 200 submissions through 2020-05-14
Saved 300 submissions through 2020-04-29
Saved 400 submissions through 2020-04-20
Saved 500 submissions through 2020-04-07
Saved 600 submissions through 2020-04-01
Saved 700 submissions through 2020-03-25
Saved 800 submissions through 2020-03-22
Saved 900 submissions through 2020-03-18
Saved 1000 submissions through 2020-03-14
Saved 1100 submissions through 2020-03-10
Saved 1200 submissions through 2020-03-06
Saved 1300 submissions through 2020-02-28
Saved 1400 submissions through 2020-02-24
Saved 1500 submissions through 2020-02-21
Saved 1600 submissions through 2020-02-11
Saved 1660 submissions through 2020-02-06
Saved 1660 submissions
Saving comments to data/reddit_italian_comments.txt
Saved 100 comments through 2021-02-01
Saved 200 comments through 2021-01-03
Saved 300 comments through 2020-11-30
Saved 400 comments through 2020-11-13
Saved 500 commen